**libraries setup**

In [ ]:
!pip install --upgrade pip
!pip install tensorflow==2.1.0

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.0.2)


In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__  # check tf version of runtime

'2.1.0'

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals  #backwards-compatibility and future-proofing

# number processing
import numpy as np 

# file handling
import os
import pathlib
from glob import glob

# archive handling
import requests
import zipfile
import tarfile

In [ ]:
tf.executing_eagerly()

True

**importing datasets as .zip starts here**

In [ ]:
url = 'http://www.iapr-tc11.org/dataset/ICDAR_SignatureVerification/SigComp2011/sigComp2011-trainingSet.zip'
r = requests.get(url, allow_redirects=True)
open('sigcomp2011.zip', 'wb').write(r.content)
#sigcomp2011

245154271

In [ ]:
url = 'http://www.iapr-tc11.org/dataset/4NSigComp2010/Dataset_4NSigComp2010.zip'
r = requests.get(url, allow_redirects=True)
open('4nsigcomp2010.zip', 'wb').write(r.content)
#4nsigcomp2010

1522758855

In [ ]:
url = 'http://www.iapr-tc11.org/dataset/4NSigComp2012/Testset_4NsigComp2012.zip'
r = requests.get(url, allow_redirects=True)
open('4nsigcomp2012.zip', 'wb').write(r.content)
#4nsigcomp2012

250615477

In [ ]:
payload = {
    'email': 'happy.quintis@gmail.com',
    'password': 'exufen'
}

with requests.Session() as s:
    p = s.post('http://tc11.cvc.uab.es/user/login', data=payload)
    # print the html returned or something more intelligent to see if it's a successful login page.
    print (p.text)

    r = s.get('http://tc11.cvc.uab.es/index.php?com=upload&action=file_down&section=dataset&section_id=114&file=132')
    open('sigwicomp2013.zip', 'wb').write(r.content)

    r = s.get('http://tc11.cvc.uab.es/index.php?com=upload&action=file_down&section=dataset&section_id=180&file=176')
    open('sigwicomp2015.zip', 'wb').write(r.content)

{ "success" : true }


**extracting datasets**

In [ ]:
# passwords
psigcomp2011 = 'I hereby accept the SigComp 2011 disclaimer.'.encode('utf-8')
p4nsigcomp2010 = 'I hereby accept the 4NSigComp2010 disclaimer.'.encode('utf-8')
p4nsigcomp2012 = 'I hereby accept the 4NsigComp2012 disclaimer.'.encode('utf-8')
psigwicomp2013 = 'IherebyaccepttheSigWiComp2013disclaimer'.encode('utf-8')
psigwicomp2015 = 'IherebyaccepttheSigWIcomp2015disclaimer'.encode('utf-8')

In [ ]:
# extract sigcomp2011
with zipfile.ZipFile("sigcomp2011.zip","r") as zip_ref:
    zip_ref.extractall("datasets/sigcomp2011", pwd=psigcomp2011)

In [ ]:
# extract 4nsigcomp2010
with zipfile.ZipFile("4nsigcomp2010.zip","r") as zip_ref:
    zip_ref.extractall("datasets/4nsigcomp2010", pwd=p4nsigcomp2010)

In [ ]:
# extract 4nsigcomp2012
with zipfile.ZipFile("4nsigcomp2012.zip","r") as zip_ref:
    zip_ref.extractall("datasets/4nsigcomp2012", pwd=p4nsigcomp2012)

In [ ]:
# extract sigwicomp2013
with zipfile.ZipFile("sigwicomp2013.zip","r") as zip_ref:
    zip_ref.extractall("datasets/sigwicomp2013", pwd=psigwicomp2013)

In [ ]:
# extract sigwicomp2015
with zipfile.ZipFile("sigwicomp2015.zip","r") as zip_ref:
    zip_ref.extractall("datasets/sigwicomp2015", pwd=psigwicomp2015)

In [ ]:
# extract more sigwicomp2015
with zipfile.ZipFile("datasets/sigwicomp2015/SigWIcomp2015-Data/Italian-Training.zip","r") as zip_ref:
    zip_ref.extractall("datasets/sigwicomp2015i", pwd=psigwicomp2015)

In [ ]:
# check filetree
currentDirectory = pathlib.Path('./datasets/sigcomp2011/trainingSet/OfflineSignatures')
a = 0
for currentFile in currentDirectory.iterdir():
    a += 1
    print(currentFile)

datasets/sigcomp2011/trainingSet/OfflineSignatures/Dutch
datasets/sigcomp2011/trainingSet/OfflineSignatures/Chinese


**convert dataset**

In [ ]:
f_2009g = glob('datasets/sigwicomp2013/TrainDutch/SigComp2009/Offline/Genuine/*/*.PNG')
f_2009f = glob('datasets/sigwicomp2013/TrainDutch/SigComp2009/Offline/Forged/*/*.PNG')

f_2010ga = glob('datasets/4nsigcomp2010/Dataset_4NSigComp2010/TrainingSet/Genuine/*.png')
f_2010ra = glob('datasets/4nsigcomp2010/Dataset_4NSigComp2010/TrainingSet/Reference/*.png')
f_2010rb = glob('datasets/4nsigcomp2010/Dataset_4NSigComp2010/TrainingSet/Reference/*.PNG')
f_2010rc = glob('datasets/4nsigcomp2010/Dataset_4NSigComp2010/TestSet/Reference/*.png')

f_2010g = f_2010ga + f_2010ra + f_2010rb + f_2010rc
f_2010f = glob('datasets/4nsigcomp2010/Dataset_4NSigComp2010/TrainingSet/Simulated/*.png')

f_2011ga = glob('datasets/sigcomp2011/trainingSet/OfflineSignatures/Dutch/TrainingSet/Offline Genuine/*.PNG')
f_2011gb = glob('datasets/sigcomp2011/trainingSet/OfflineSignatures/Dutch/TrainingSet/Offline Genuine/*.png')

f_2011g = f_2011ga + f_2011gb
f_2011f = glob('datasets/sigcomp2011/trainingSet/OfflineSignatures/Dutch/TrainingSet/Offline Forgeries/*.png')

f_2012g = glob('datasets/4nsigcomp2012/Testset_4NsigComp2012/Ref/*/*.PNG')

In [ ]:
len(f_2009f)

1838

In [ ]:
a_2009g = [tf.io.read_file(f) for f in f_2009g]
a_2009f = [tf.io.read_file(f) for f in f_2009f]

a_2010g = [tf.io.read_file(f) for f in f_2010g]
a_2010f = [tf.io.read_file(f) for f in f_2010f]

a_2011g = [tf.io.read_file(f) for f in f_2011g]
a_2011f = [tf.io.read_file(f) for f in f_2011f]

a_2012g = [tf.io.read_file(f) for f in f_2012g]

**variables**

In [ ]:
IMG_HEIGHT = 775
IMG_WIDTH = 224
CLASS_NAMES = np.array(["Genuine","Forged"])
BATCH_SIZE = 30
EPOCH_SIZE = 10

**convert to numpy**

In [ ]:
def setupImg(array): 
  for i in range(len(array)):
    oof = array[i]
    oof = tf.image.decode_png(oof, channels=1)
    oof = tf.image.convert_image_dtype(oof, tf.float32)
    oof = tf.image.resize(oof, [IMG_WIDTH, IMG_HEIGHT])
    array[i] = oof.numpy()
  return array

In [ ]:
a_g = a_2009g+a_2010g+a_2011g+a_2012g+a_2015i
a_g = np.array(setupImg(a_g)) # decodes genuines
a_g.shape

(716, 224, 775, 1)

In [ ]:
a_f = a_2009f+a_2010f+a_2011f
a_f = np.array(setupImg(a_f)) # decodes forgeries
a_f.shape

(2065, 224, 775, 1)

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
np.savez("/content/drive/My Drive/a_rs.npz", a_g=a_g, a_f=a_f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**stop here**